In [244]:
import comtypes.client
import random

In [245]:

# 连接到运行中的 SAP2000
sap_object = comtypes.client.GetActiveObject("CSI.SAP2000.API.SapObject")
SapModel = sap_object.SapModel

# 解锁模型
SapModel.SetModelIsLocked(False)

# 设置单位
SapModel.SetPresentUnits(9)  # 设定单位为 N/mm^2


0

In [246]:
def random_goods_distribution(num_z, num_x, num_y):
    # 创建一个 num_z 行，num_x * num_y 列的矩阵
    distribution_matrix = []
    
    for layer in range(num_z):
        # 随机生成当前层的货物分布，0 或 1
        distribution = [random.choice([0, 1]) for _ in range(num_x * num_y)]
        distribution_matrix.append(distribution)  # 将当前层的分布添加到矩阵中
        
    return distribution_matrix



In [247]:
# 循环进行随机货位分布并计算频率
num_trials = 10 # 模拟次数
surface_pressure_area = 1000000*9.8 / (2000*2000)  # 均布面力，可以根据需要调整
num_z = 3
num_x = 7
num_y = 7



In [248]:
#根据货位分布矩阵，对结构施加均布面力，然后进行模态分析
num_loc = 147  # 货位总数
for trial in range(num_trials):
    # 随机分布货位
    distribution_matrix = random_goods_distribution(num_z, num_x, num_y)
    
    print(f"Trial {trial + 1}: Goods Distribution Matrix:")
    for layer in distribution_matrix:
        print(layer)
    
    flattened_distribution = [item for sublist in distribution_matrix for item in sublist]

    # 在 SAP2000 中添加均布面力
    for loc in range(num_loc):
        if flattened_distribution[loc] == 1:
            area_name = str(loc + 1)
            
            ret=sap_object.SapModel.AreaObj.SetLoadUniform(area_name,
                                                       "DEAD",
                                                       surface_pressure_area,
                                                       10,
                                                       True,
                                                       "Global",
                                                       0)
            #ret = sap_object.SapModel.AreaObj.SetLoadSurfacePressure(area_name, "DEAD", -2, 2450, " ", False, 0)
            #if ret != 0:
                #print("Area load not assigned successfully.")
            #else:
                #print("Area load assigned successfully.")
            

    #执行模态分析
    ret=SapModel.Analyze.RunAnalysis()

    #ret = SapModel.Analyze.RunAnalysis()
    #if ret != 0:
        #print("Analysis did not run successfully.")
    #else:
        #print("Analysis run successfully.")

    NumberResults = 12
    LoadCase = []
    StepType = []
    StepNum = []
    Period = []
    Frequency = []
    CircFreq = []
    EigenValue = []

    # 调用函数获取模态数据
    result = SapModel.Results.ModalPeriod(NumberResults, 
                                          LoadCase, 
                                          StepType, 
                                          StepNum, 
                                          Period, 
                                          Frequency, 
                                          CircFreq, 
                                          EigenValue)
    print(result[6])  # 6 代表CircFreq


     #解锁模型
    SapModel.SetModelIsLocked(False)


    # 删除分析结果

    ret=sap_object.SapModel.Analyze.DeleteResults(" ", True)

    #ret = sap_object.SapModel.Analyze.DeleteResults(" ", True)
    #if ret != 0:
        #print("Results not deleted successfully.")
    #else:
        #print("Results deleted successfully.")

    # 删除均布面力
    for load in range(num_loc):
        if flattened_distribution[load] == 1:
            load_name = str(load + 1)

            ret=sap_object.SapModel.AreaObj.DeleteLoadUniform(load_name, 
                                                          "DEAD", 
                                                          0)

            #ret= sap_object.SapModel.AreaObj.DeleteLoadUniform(load_name, "DEAD", 0)
            #if ret != 0:
                #print("Area load not deleted successfully.")
            #else:
                #print("Area load deleted successfully.")

    #解锁模型
    SapModel.SetModelIsLocked(False)

# 关闭 SAP2000
#sap_object.ApplicationExit(False)

Trial 1: Goods Distribution Matrix:
[1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0]
[1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
(1.3432065532048838, 1.3856161716542226, 1.4615299848174312, 1.498321904953425, 1.7752719568041835, 1.7823047744325806, 2.041617115957163, 2.1174740179412224, 2.3973451760946562, 2.4147791341760954, 2.6699750442447625, 2.766522803863219)
Trial 2: Goods Distribution Matrix:
[0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1]
[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1